# Анализ базы данных

На волне возросшего интереса людей к чтению книг компания решила купить крупный сервис для чтения книг по подписке. Необходимо изучить базу данных.

**Цель** - проанализировать базу данных. Полученная информация поможет сформулировать ценностное предложение для нового продукта.  

**Задачи**:
- Посчитать, сколько книг вышло после 1 января 2000 года
- Для каждой книги посчитать количество обзоров и среднюю оценку
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры
- Определить автора с самой высокой средней оценкой книг — учитывать необходимо только книги с 50 и более оценками
- Посчитайтать среднее количество обзоров от пользователей, которые поставили больше 50 оценок  

**Описание базы данных**:  

Таблица books:
- book_id — идентификатор книги
- author_id — идентификатор автора
- title — название книги
- um_pages — количество страниц
- publication_dat — дата публикации книги
- publisher_id — идентификатор издателя

Таблица authors:
- author_id — идентификатор автора
- author — имя автора

Таблица publishers:
- publisher_id — идентификатор издательства
- publisher — название издательства

Таблица ratings:
- rating_id — идентификатор оценки
- book_id — идентификатор книги
- username — имя пользователя, оставившего оценку
- rating — оценка книги

Таблица reviews:
- review_id — идентификатор обзора
- book_id — идентификатор книги
- username — имя пользователя, написавшего обзор
- text — текст обзора  

**Содержание**:
1. <a href='#Импорт-библиотек-и-подключение-к-БД'>Импорт библиотек и подключение к БД</a>
2. <a href='#Исследование-таблиц'>Исследование таблиц</a>
3. <a href='#Выполнение-запросов-к-БД'>Выполнение запросов к БД</a>
4. <a href='#Выводы'>Выводы</a>

### Импорт библиотек и подключение к БД

Импорт библиотек

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Создание подключения к базе данных

In [2]:
db_config = {
    'user': 'praktikum_student',
    'pwd': 'Sdf4$2;d-d30pp',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,
    'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Создание функции для выполнения запросов.

In [3]:
def query(engine):
    
    def execute(query):
        return pd.io.sql.read_sql(query, con = engine)
    
    return execute

In [4]:
execute_query = query(engine)

### Исследование таблиц

Исследуем таблицы. Выведем первые строки всех таблиц.

In [5]:
print('Первые строки')
for table in ['books', 'authors', 'ratings', 'reviews', 'publishers']:
    query = f"SELECT * FROM {table} LIMIT 2"
    print(f'Таблица {table}:')
    display(execute_query(query))
    print('--------------')

Первые строки
Таблица books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


--------------
Таблица authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs


--------------
Таблица ratings:


...

--------------
Таблица reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...


--------------
Таблица publishers:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book


--------------


### Выполнение запросов к БД

Посчитаем, сколько книг вышло после 1 января 2000 года

In [6]:
query = "SELECT COUNT(book_id) FROM books WHERE publication_date > '2000-01-01'"

execute_query(query)

,count
0,819


После 1 января 200 года вышло 819 книг.  

Для каждой книги посчитаем количество обзоров и среднюю оценку

In [7]:
query = '''
    SELECT title, AVG(rating), COUNT(review_id) FROM books 
    INNER JOIN ratings ON books.book_id = ratings.book_id
    LEFT JOIN reviews ON books.book_id = reviews.book_id
    GROUP BY books.title
'''

books_ratings = execute_query(query)
books_ratings.columns = ['book_name', 'avg_rating', 'reviews_count']
books_ratings

,book_name,avg_rating,reviews_count
0,The Count of Monte Cristo,4.217391,115
1,Count Zero (Sprawl #2),2.500000,4
2,The Botany of Desire: A Plant's-Eye View of th...,3.500000,4
3,The Poisonwood Bible,4.363636,110
4,The Canterbury Tales,3.333333,18
...,...,...,...
994,Of Love and Other Demons,4.500000,4
995,In the Heart of the Sea: The Tragedy of the Wh...,3.333333,9
996,Welcome to Temptation (Dempseys #1),5.000000,4
997,World's End (The Sandman #8),4.500000,4


Получили датафрейм, содержащий количество обзоров и среднюю оценку для каждой книги.  

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [8]:
query = '''
    SELECT publisher, COUNT(subquery.book_id) FROM publishers
    LEFT JOIN (
        SELECT book_id, publisher_id FROM books WHERE num_pages > 50
    ) AS subquery ON publishers.publisher_id = subquery.publisher_id
    GROUP BY publishers.publisher
    ORDER BY COUNT(subquery.book_id) DESC
    LIMIT 1
'''

execute_query(query)

,publisher,count
0,Penguin Books,42


Больше всего книг выпустило издательство "Penguin Books". Оно выпустило 42 книги.  

Определим автора с самой высокой средней оценкой книг — учтём только книги с 50 и более оценками

In [9]:
query = '''
    SELECT author, AVG(avg) FROM books 
    RIGHT JOIN
    (SELECT book_id, COUNT(book_id), AVG(rating) 
    FROM ratings 
    GROUP BY book_id 
    HAVING COUNT(book_id) > 50) AS subquery
    ON books.book_id = subquery.book_id
    LEFT JOIN authors ON authors.author_id = books.author_id
    GROUP BY author
    ORDER BY avg DESC
'''

execute_query(query).head(1)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Самую высокую среднюю оценку имеют книги автора J.K. Rowling/Mary GrandPré.  

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [10]:
query = '''
    SELECT AVG(count) FROM
    (SELECT reviews.username, COUNT(review_id) FROM reviews RIGHT JOIN 
    (SELECT username FROM ratings GROUP BY username HAVING COUNT(rating_id) > 50) AS subquery
    ON reviews.username = subquery.username
    GROUP BY reviews.username) AS subquery2
'''

execute_query(query)

,avg
0,24.333333


Те, кто поставили больше 50 оценок, в среднем оставляют 24.3 обзора.

### Выводы

- После 1 января 200 года вышло 819 книг
- Получили датафрейм, содержащий количество обзоров и среднюю оценку для каждой книги
- Больше всего книг выпустило издательство "Penguin Books". Оно выпустило 42 книги
- Самую высокую среднюю оценку имеют книги автора J.K. Rowling/Mary GrandPré
- Те, кто поставили больше 50 оценок, в среднем оставляют 24.3 обзора